In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from scipy import stats as st
import matplotlib.pyplot as plt
from PIL import Image
import scipy

In [2]:
def h(x):
    # Activation function, let's just use Relu
    return np.maximum(0,x)

def convolve(kernels_r, kernels_g, kernels_b, x_pixels, y_pixels, image_data, print_names = []):
    x_pixels_convolved = x_pixels - kernels_r[0].shape[0] + 1 # new x pixels
    y_pixels_convolved = y_pixels - kernels_r[0].shape[1] + 1 # new y pixels
    channels_convolved = kernels_r.shape[0]                   # how many kernels and, hence, output channels

    # of original image:
    selected_image_r = np.empty((x_pixels,y_pixels)) # red channel
    selected_image_g = np.empty((x_pixels,y_pixels)) # blue channel
    selected_image_b = np.empty((x_pixels,y_pixels)) # green channel

    index = 0
    for i in range(x_pixels):
        for j in range(y_pixels):
            selected_image_r[i][j] = 255*image_data[index+0]
            selected_image_g[i][j] = 255*image_data[index+1]
            selected_image_b[i][j] = 255*image_data[index+2]
            index += 3

    convolved_image = np.empty((channels_convolved,x_pixels_convolved,y_pixels_convolved))
    # now let's cycle through each channel and apply appropriate kernels
    for c in range(channels_convolved):
        convolved_image_r = scipy.signal.convolve2d(selected_image_r,kernels_r[c],mode='valid') # red channel
        convolved_image_g = scipy.signal.convolve2d(selected_image_g,kernels_g[c],mode='valid') # blue channel
        convolved_image_b = scipy.signal.convolve2d(selected_image_b,kernels_b[c],mode='valid') # green channel
            
        for i in range(x_pixels_convolved):
            for j in range(y_pixels_convolved):
                convolved_image[c][i][j] = h(convolved_image_r[i][j] + convolved_image_g[i][j] + convolved_image_b[i][j] + 1)  

        if len(print_names) > 0:
            cm = plt.get_cmap('bwr')
            colored_image = cm(convolved_image[c]/255)
            image = Image.fromarray((colored_image[:, :, :3] * 255).astype(np.uint8)).save(print_names[c])
        
    return convolved_image

def max_pool(input_image, windowsize, print_names = []):
    x_pixels = int(input_image[0].shape[0]/windowsize) # new x pixels
    y_pixels = int(input_image[0].shape[1]/windowsize) # new y pixels
    channels = int(input_image.shape[0])     # channels
    
    max_pooled_image = np.zeros((channels,x_pixels,y_pixels))
    # now let's cycle through each channel and max pool
    for c in range(channels):
        for i in range(x_pixels):
            for j in range(y_pixels):
                sum = np.zeros((windowsize,windowsize))
                for i2 in range(windowsize):
                    for j2 in range(windowsize):
                        sum[i2][j2] = input_image[c][windowsize*i + i2][windowsize*j + j2]
                max_pooled_image[c][i][j] = np.max(sum)

        if len(print_names) > 0:
            cm = plt.get_cmap('bwr')
            colored_image = cm(max_pooled_image[c]/255)
            image = Image.fromarray((colored_image[:, :, :3] * 255).astype(np.uint8)).save(print_names[c])

    return max_pooled_image

In [3]:
from sklearn.datasets import fetch_lfw_people
color = True
people = fetch_lfw_people(min_faces_per_person=8, resize=1, color=color)

x = people.data 
y = people.target

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75)

print("Total data points, x pixels, y pixels, color channels")
print(people.images.shape)
print(x_test.shape)
print(x_train.shape)

Total data points, x pixels, y pixels, color channels
(4503, 125, 94, 3)
(3377, 35250)
(1126, 35250)


In [20]:
# Let's define some convolutions
x_pixels = people.images.shape[1]
y_pixels = people.images.shape[2]
windowsize = 5
number_of_kernels = 3
kernels_r = np.empty((number_of_kernels,3,3))
kernels_g = np.empty((number_of_kernels,3,3))
kernels_b = np.empty((number_of_kernels,3,3))
# first kernel
kernels_r[0] = np.array([[1,1,1],
                         [0,0,0],
                         [-1, -1, -1]])
kernels_g[0] = np.array([[1,1,1],
                         [0,0,0],
                         [-1, -1, -1]])
kernels_b[0] = np.array([[1,1,1],
                         [0,0,0],
                         [-1, -1, -1]])
# second kernel
kernels_r[1] = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]])
kernels_g[1] = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]])
kernels_b[1] = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]])
# third kernel
kernels_r[2] = np.array([[-1, -1, -1],
                         [-1, 8, -1],
                         [-1, -1, -1]])
kernels_g[2] = np.array([[-1, -1, -1],
                         [-1, 8, -1],
                         [-1, -1, -1]])
kernels_b[2] = np.array([[-1, -1, -1],
                         [-1, 8, -1],
                         [-1, -1, -1]])

In [21]:
new_size = (max_pool(convolve(kernels_r, kernels_g, kernels_b, x_pixels, y_pixels, x_train[0]), windowsize).shape)
print(new_size)

x_train_new = np.empty((x_train.shape[0],(new_size[0]*new_size[1]*new_size[2])))
x_test_new = np.empty((x_test.shape[0],(new_size[0]*new_size[1]*new_size[2])))

print("Starting testing data")
for i in range(len(x_test)):
    if i % 100 ==0:
        print(i)
    convolved_image = convolve(kernels_r, kernels_g, kernels_b, x_pixels, y_pixels, x_test[i])
    x_test_new[i] = np.ravel(max_pool(convolved_image, windowsize))
print("Starting training data")
for i in range(len(x_train)):
    if i % 100 ==0:
        print(i)
    convolved_image = convolve(kernels_r, kernels_g, kernels_b, x_pixels, y_pixels, x_train[i])
    x_train_new[i] = np.ravel(max_pool(convolved_image, windowsize))

print("After convolutions:")
print(x_train_new.shape)
print(x_test_new.shape)

(3, 24, 18)
Starting testing data
0
100
200
300
400
500
600
700
800
900
1000
1100
Starting training data
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
After convolutions:
(3377, 1296)
(1126, 1296)


In [22]:
# kNN
k=100
model = NearestNeighbors(n_neighbors=k)
model.fit(x_train_new,y_train)
test_neighbors = model.kneighbors_graph(x_test_new)
print("Picking " + str(k) + " nearest neighbors, a total of " + str(x_train_new.shape[0]) + " training points with " + str(x_train_new.shape[1]) + " dimensions")

Picking 100 nearest neighbors, a total of 3377 training points with 1296 dimensions


In [23]:
# test_nearest_neighbors looks through test_neighbors and gets the classifications of the k nearest ones
test_nearest_neighbors = np.empty((y_test.shape[0],k))
# y_pred gets the mode of the nearest neighbors
y_pred = np.empty((y_test.shape[0],1))

print("Total number of test points: " + str(x_test.shape[0]))
for i in range(x_test_new.shape[0]):
    if i%50 == 0:
        print("i=" + str(i))
    test_nearest_neighbors[i] = y_train[np.nonzero(test_neighbors.toarray()[i])]
    y_pred[i] = int(st.mode(test_nearest_neighbors[i], keepdims=True).mode)

print("Done!")

Total number of test points: 1126
i=0
i=50
i=100
i=150
i=200
i=250
i=300
i=350
i=400
i=450
i=500
i=550
i=600
i=650
i=700
i=750
i=800
i=850
i=900
i=950
i=1000
i=1050
i=1100
Done!


In [24]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("Accuracy: " + str(np.round(100*conf_matrix.trace()/conf_matrix.sum(),2)) + "%")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 17.23%
